In [6]:
import robotic as ry
import json
import numpy as np
from config import setup_config, startup_robot
from random_paths import run_waypoints_one_by_one, pushPCpoint, computeKomo
from visual import lookAtObj, getPushPoint, getObject
from arena import *
import random

verbose = 0

INITIAL_OBJ_POS = [-.5, 0, .69]
DEBUG = False
OBJ_HEIGHT = .08

ON_REAL = False
NUMBER_OF_PUSH_TRIALS=100
table_width=.89
table_height=.55
table_middle=np.array([-.23, -.16, .651])

innerR= .29
robot_pos = np.array([-.03, -.24, .651])

In [7]:
#-- load parameters, typically automatically from 'rai.cfg'
ry.params_print()

#-- define a configuration
C = setup_config(INITIAL_OBJ_POS, ON_REAL)
if DEBUG:
    key = C.view(verbose>0, 'happy with the config?')
    print('key pressed: ', chr(key)) #use this for basic interaction, e.g. aborting the program
    if chr(key)=='q':
        exit()

bot = startup_robot(C, ON_REAL)

data = []
obj_pos = INITIAL_OBJ_POS

# Generate Arena
arena = RectangularArena(middleP=table_middle, width=table_width, height=table_height, middlePCirc=robot_pos, innerR=innerR)
arena.plotArena(C)

-- module.cpp:operator():95(0) python,
RealSense/resolution: 640,
RealSense/alignToDepth!,
physx/motorKp: 10000,
physx/motorKd: 1000,
physx/multiBody!,
bot/useGripper,
bot/useRobotiq!,
bot/useArm: left,
bot/blockRealRobot!,
botsim/hyperSpeed: 1,
botsim/verbose: 1,
botsim/engine: physx,
physx/verbose: 1,
physx/yGravity!,
physx/softBody!,
physx/multiBodyDisableGravity,
physx/jointedBodies!,
physx/angularDamping: 0.1,
physx/defaultFriction: 1,
physx/defaultRestitution: 0.1,
bot/useOptitrack!,
bot/useAudio!,
bot/raiseWindow!,
KOMO/verbose: 1,
KOMO/animateOptimization: 0,
KOMO/mimicStable,
KOMO/useFCL,
KOMO/unscaleEqIneqReport!,
KOMO/sampleRate_stable: 0,
opt/verbose: 1,
opt/stopTolerance: 0.01,
opt/stopFTolerance: -1,
opt/stopGTolerance: -1,
opt/stopEvals: 1000,
opt/stopIters: 1000,
opt/stopOuters: 1000,
opt/stopLineSteps: 10,
opt/stopTinySteps: 10,
opt/initStep: 1,
opt/minStep: -1,
opt/maxStep: 0.2,
opt/damping: 1,
opt/stepInc: 1.5,
opt/stepDec: 0.5,
opt/wolfe: 0.01,
opt/boundedNewton,
op

In [8]:
# Point towards set initial object position
lookAtObj(bot, C, np.array(obj_pos))
push_points  = getPushPoint(bot, C, arena)

{ time: 0.001191, evals: 9, done: 1, feasible: 1, sos: 0.0102831, f: 0, ineq: 0, eq: 0.00346971 }


In [9]:
for i in range(NUMBER_OF_PUSH_TRIALS):

    if not obj_pos: break
    key=bot.sync(C, .1)
    if chr(key) == "q":
        print("Terminated (main loop)")
        break

    lookAtObj(bot, C, np.array(obj_pos))
    obj_pos = getObject(bot, C, arena)
    push_points  = getPushPoint(bot, C, arena)

    while True:
        pushP = random.choice(push_points)
        bot.sync(C, .1)

        #-- compute a motion (debug this inside the method)
        waypoints = pushPCpoint(pushP[0], pushP[1], robot_pos)

        # if not success: break
        path, feasible = computeKomo(C, waypoints)

        #-- now, if we're happy with the motion, send it to the robot
        if feasible:
            run_waypoints_one_by_one(bot, path, True, C)
            break

{ time: 0.001433, evals: 3, done: 1, feasible: 1, sos: 0.0051003, f: 0, ineq: 0, eq: 0.00511817 }
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'pcl'
====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
{ time: 0.814798, evals: 99, done: 1, feasible: 0, sos: 0, f: 0, ineq: 0, eq: 0.963715 }
{ time: 0.471731, evals: 73, done: 1, feasible: 0, sos: 0, f: 0, ineq: 0, eq: 0.589643 }
{ time: 0.585062, evals: 89, done: 1, feasible: 0, sos: 0, f: 0, ineq: 0, eq: 0.619221 }
==nlp== it:   0  evals:  13  A(x):    0.36351  f:          0  g:          0  h:    2.11929  |x-x'|:     1.6913 	stop:DeltaConverge
==nlp== it:   1  evals:  13  A(x):    2.54457  mu:5
==nlp== it:   1  evals:  82  A(x):    1.09264  f:          0  g:          0  h:   0.963716  |x-x'|:    2.00976 	stop:TinyXSteps (bad:1)
==nlp== it:   2  evals:  82  A(x):    5.08209  mu:25
==nlp== it:   2  evals:  99  A(x):    5.08209  f:          0  g:          0  h:   0.963715  |x-x'|:1.3673

IndexError: Cannot choose from an empty sequence

In [ ]:
bot.home(C)
json.dump(data, open('data/data.json', 'w'))

In [ ]:
del bot
del C